In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [132]:
img = cv.imread('noisy rect.png',0)
img_orig = img.copy()
template_orig = cv.imread('lech.png',0)
template_orig = cv.imread('lech-simple.png',0)
#template_orig = cv.imread('line.png',0)
#template_orig = cv.imread('kh.png',0)
template_orig = cv.imread('lamed.png',0)

In [ ]:
from PIL import Image, ImageDraw
import gpxpy as mod_gpxpy
from PIL import Image as mod_pil_image
from PIL import ImageDraw as mod_pil_draw
import glob as mod_glob
import math as mod_math
import numpy
osm_tile_res = 256

def format_time(time_s):
    if not time_s:
        return 'n/a'
    minutes = mod_math.floor(time_s / 60.)
    hours = mod_math.floor(minutes / 60.)
    return '%s:%s:%s' % (str(int(hours)).zfill(2), str(int(minutes % 60)).zfill(2), str(int(time_s % 60)).zfill(2)) 

def get_map_suffix ():
#    return "osm-cycle"
#    return "seznam-turist"
    return "roads"


def osm_lat_lon_to_x_y_tile (lat_deg, lon_deg, zoom):
    """ Gets tile containing given coordinate at given zoom level """
    # taken from http://wiki.openstreetmap.org/wiki/Slippy_map_tilenames, works for OSM maps and mapy.cz
    lat_rad = mod_math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - mod_math.log(mod_math.tan(lat_rad) + (1 / mod_math.cos(lat_rad))) / mod_math.pi) / 2.0 * n)
    return (xtile, ytile)


def osm_get_auto_zoom_level ( min_lat, max_lat, min_lon, max_lon, max_n_tiles):
    """ Gets zoom level which contains at maximum `max_n_tiles` """
    for z in range (0,17):
        x1, y1 = osm_lat_lon_to_x_y_tile (min_lat, min_lon, z)
        x2, y2 = osm_lat_lon_to_x_y_tile (max_lat, max_lon, z)
        max_tiles = max (abs(x2 - x1), abs(y2 - y1))
        if (max_tiles > max_n_tiles):
            print ("Max tiles: %d" % max_tiles)
            return z 
    return 17


class MapCreator:
    """ Class for map drawing """

    def __init__(self, min_lat, max_lat, min_lon, max_lon, z):
        """ constructor """
        x1, y1 = osm_lat_lon_to_x_y_tile (min_lat, min_lon, z)
        x2, y2 = osm_lat_lon_to_x_y_tile (max_lat, max_lon, z)
        self.x1 = min (x1, x2)
        self.x2 = max (x1, x2)
        self.y1 = min (y1, y2)
        self.y2 = max (y1, y2)
        self.w = (self.x2 - self.x1 + 1) * osm_tile_res
        self.h = (self.y2 - self.y1 + 1) * osm_tile_res
        self.z = z
        print (self.w, self.h)
        self.dst_img = mod_pil_image.new ("RGB", (self.w, self.h), 'white')

    def lat_lon_to_image_xy (self, lat_deg, lon_deg):
        """ Internal. Converts lat, lon into dst_img coordinates in pixels """
        lat_rad = mod_math.radians(lat_deg)
        n = 2.0 ** self.z
        xtile_frac = (lon_deg + 180.0) / 360.0 * n
        ytile_frac = (1.0 - mod_math.log(mod_math.tan(lat_rad) + (1 / mod_math.cos(lat_rad))) / mod_math.pi) / 2.0 * n
        img_x = int( (xtile_frac-self.x1)*osm_tile_res )
        img_y = int( (ytile_frac-self.y1)*osm_tile_res )
        return (img_x, img_y)

    def draw_track (self, gpx):
        """ Draw GPX track onto map """
        draw = mod_pil_draw.Draw (self.dst_img)
        trk = 0         # Just changes color of segment a little
        for track in gpx.tracks:
            for segment in track.segments:
                idx = 0
                x_from = 0
                y_from = 0
                for point in segment.points:
                    if (idx == 0):
                        x_from, y_from = self.lat_lon_to_image_xy (point.latitude, point.longitude)
                    else:
                        x_to, y_to = self.lat_lon_to_image_xy (point.latitude, point.longitude)
#                        draw.line ((x_from,y_from,x_to,y_to), (255,0,trk), 2)
                        draw.line ((x_from,y_from,x_to,y_to), 'black', 4)
                        x_from = x_to
                        y_from = y_to
                    idx += 1
                trk += 32
                if (trk > 160):
                    trk = 0
                    
    def save_image(self, filename):
        print("Saving " + filename) 
        self.dst_img.save (filename)


In [141]:
#gpx_files = mod_glob.glob (r"*.gpx")
gpx_files = ['test2.gpx']

In [142]:
for gpx_file in gpx_files:
        try:
            gpx = mod_gpxpy.parse(open(gpx_file))

            # Print some track stats
            print ('--------------------------------------------------------------------------------')
            print ('  GPX file     : %s' % gpx_file)
            start_time, end_time = gpx.get_time_bounds()
            print('  Started       : %s' % start_time)
            print('  Ended         : %s' % end_time)
            print('  Length        : %2.2fkm' % (gpx.length_3d() / 1000.))
            moving_time, stopped_time, moving_distance, stopped_distance, max_speed = gpx.get_moving_data()
            print('  Moving time   : %s' % format_time(moving_time))
            print('  Stopped time  : %s' % format_time(stopped_time))
#            print('  Max speed     : %2.2fm/s = %2.2fkm/h' % (max_speed, max_speed * 60. ** 2 / 1000.))    
            uphill, downhill = gpx.get_uphill_downhill()
            print('  Total uphill  : %4.0fm' % uphill)
            print('  Total downhill: %4.0fm' % downhill)
            min_lat, max_lat, min_lon, max_lon = gpx.get_bounds()
            print("  Bounds        : [%1.4f,%1.4f,%1.4f,%1.4f]" % (min_lat, max_lat, min_lon, max_lon))
            z = osm_get_auto_zoom_level (min_lat, max_lat, min_lon, max_lon, 6)
            print("  Zoom Level    : %d" % z)


            # Create the map
            map_creator = MapCreator (min_lat, max_lat, min_lon, max_lon, z)
            #map_creator.cache_area()
            #map_creator.create_area_background()
            map_creator.draw_track(gpx)
            map_creator.save_image (gpx_file[:-4] + '-' + get_map_suffix() + '.png')
            
        except Exception as e:
            mod_logging.exception(e)
            print('Error processing %s' % gpx_file)
            mod_sys.exit(1) 

pil_image = map_creator.dst_img#.convert('RGB') 
open_cv_image = numpy.array(pil_image) 
# Convert RGB to BGR 
map_orig = cv.cvtColor(numpy.array(pil_image), cv.COLOR_RGB2GRAY)
map_orig.shape


--------------------------------------------------------------------------------
  GPX file     : test2.gpx
  Started       : None
  Ended         : None
  Length        : 33.31km
  Moving time   : n/a
  Stopped time  : n/a
  Total uphill  :  665m
  Total downhill:  682m
  Bounds        : [32.7326,32.7789,34.9892,35.0535]
Max tiles: 12
  Zoom Level    : 16
3328 2816
Saving test2-roads.png


(2816, 3328)

In [152]:
w, h = template_orig.shape[::-1]
myres = []
#for scale_percent in [0.1, 0.2, 0.5, 1.0, 5.0, 10.0]:
#for scale_percent in [50, 100, 200, 400, 800]:
#for scale_percent in [10, 20, 40, 60, 80]:
for scale_percent in np.arange(5, 40, 5):
#for scale_percent in [50]:
    width = int(w * scale_percent / 100)
    height = int(h * scale_percent / 100)
    if(width > 0 and height > 0):
        dim = (width, height)
        template = cv.resize(template_orig, dim, interpolation = cv.INTER_AREA)
        t = template
        t[template == 0] = 0
        t[template > 0] = 1
        mask = t

        template = 255 - template # invert
        #template = cv.Canny(template, 50, 200)
        # All the 6 methods for comparison in a list
#        methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR', 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
        methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED']
#        methods = ['cv.TM_SQDIFF']
        #methods = ['cv.TM_CCOEFF']
        for meth in methods:
            # print (method, width, height)
            # img = img_orig.copy()
            img = map_orig.copy()
            img = 255 - img  # invert
            #img = cv.Canny(img, 50, 200)
            method = eval(meth)
            # Apply template Matching
            res = cv.matchTemplate(img, template, method) # , mask=mask)
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
            # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
            if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
                top_left = min_loc
            else:
                top_left = max_loc
            if(max(top_left) > 0):
                res_tuple = (top_left[0], width, top_left[1], height, res, meth, min_val, max_val)
                myres.append(res_tuple)

In [ ]:
mm = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR', 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
if(len(myres) > 0):
    f, axs = plt.subplots(1,1,figsize=(100,100))
    axs.set_xlim(0, img.shape[0])
    axs.set_ylim(img.shape[1], 0)
    axs.imshow(img, cmap = 'gray')
    for r in myres:
        tlx, width, tly, height, res, method, minval, maxval = r
        if(True or method == mm[5]):
            top_left = (tlx, tly)
            bottom_right = (tlx + width, tly + height)
            #cv.rectangle(img, top_left, bottom_right, 128, cv.LINE_4)
            extent = [tlx, tlx+width, tly+height, tly]
            axs.imshow(template, cmap='spring', extent=extent, alpha=0.5)
            #axs.text(0, 0, method, {'weight' : 'bold', 'size': 96})
    plt.show()

In [147]:
list(map(lambda x : (x[5], x[6], x[7]), myres))
#help(cv.imread)

[]